<a href="https://colab.research.google.com/github/LucjanSakowicz/data-science-projects/blob/main/titanic/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Load data

In [131]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

Print basic train data info

In [51]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Print first few rows.

In [53]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [134]:
df_train = df_train.append(df_test)

<ipython-input-134-ea921166dab2>:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Name column seems widely unorganized, we'll try to fix that.

First will extract part before coma, which looks like surname.

In [135]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [136]:
df_train[['Surname', 'FirstNames']] = df_train['Name'].str.split(',', expand=True)
df_train[['Surname', 'FirstNames']]

,Surname,FirstNames
0,Braund,Mr. Owen Harris
1,Cumings,Mrs. John Bradley (Florence Briggs Thayer)
2,Heikkinen,Miss. Laina
3,Futrelle,Mrs. Jacques Heath (Lily May Peel)
4,Allen,Mr. William Henry
...,...,...
413,Spector,Mr. Woolf
414,Oliva y Ocana,Dona. Fermina
415,Saether,Mr. Simon Sivertsen
416,Ware,Mr. Frederick


Split First names into title, Names and others.

In [137]:
df_train[['Title', 'Names', 'Other']] = df_train['FirstNames'].str.split('.', expand=True)
df_train.drop(columns=['FirstNames', 'Name'], inplace=True)

In [138]:
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names,Other
0,1,0.0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Braund,Mr,Owen Harris,None
1,2,1.0,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Cumings,Mrs,John Bradley (Florence Briggs Thayer),None
2,3,1.0,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Heikkinen,Miss,Laina,None
3,4,1.0,1,female,35.0,1,0,113803,53.1000,C123,S,Futrelle,Mrs,Jacques Heath (Lily May Peel),None
4,5,0.0,3,male,35.0,0,0,373450,8.0500,NaN,S,Allen,Mr,William Henry,None


What exactly is in Other column?

In [139]:
df_train['Other'].value_counts()

 Barrett)    1
Name: Other, dtype: int64

In [140]:
df_train[df_train['Other'].isna() == False]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names,Other
513,514,1.0,1,female,54.0,1,0,PC 17603,59.4,NaN,C,Rothschild,Mrs,Martin (Elizabeth L,Barrett)


Remove non-required column

In [141]:
df_train.iloc[513, -2] = df_train.iloc[513, -2] + df_train.iloc[513,-1]
df_train.drop(columns='Other', inplace=True)
print(df_train)

     PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch  \
0              1       0.0       3    male  22.0      1      0   
1              2       1.0       1  female  38.0      1      0   
2              3       1.0       3  female  26.0      0      0   
3              4       1.0       1  female  35.0      1      0   
4              5       0.0       3    male  35.0      0      0   
..           ...       ...     ...     ...   ...    ...    ...   
413         1305       NaN       3    male   NaN      0      0   
414         1306       NaN       1  female  39.0      0      0   
415         1307       NaN       3    male  38.5      0      0   
416         1308       NaN       3    male   NaN      0      0   
417         1309       NaN       3    male   NaN      1      1   

                 Ticket      Fare Cabin Embarked        Surname    Title  \
0             A/5 21171    7.2500   NaN        S         Braund       Mr   
1              PC 17599   71.2833   C85        C       

Trim strings from blank characters

In [142]:
df_train[['Surname', 'Title', 'Names']] = df_train[['Surname', 'Title', 'Names']].apply(lambda x: x.str.strip())

In [143]:
df_train['Surname'].value_counts()

Andersson    11
Sage         11
Goodwin       8
Asplund       8
Davies        7
             ..
Milling       1
Maisner       1
Goncalves     1
Campbell      1
Saether       1
Name: Surname, Length: 875, dtype: int64

Let's make bar chart for better visualisation of surnames

In [144]:
px.bar(x=df_train['Surname'].value_counts().index, y=df_train['Surname'].value_counts(), template='plotly_dark', title='Surname occurrences',
       width=950, color_discrete_sequence=['#03fcb5'])

Some of the Surnames looks like have more than one word and some have characters which are not letters, let's take a look

In [145]:
temp_surname = df_train['Surname'].str.split(' ', expand=True).apply(lambda x: x.str.strip())
print(temp_surname)

             0     1      2
0       Braund  None   None
1      Cumings  None   None
2    Heikkinen  None   None
3     Futrelle  None   None
4        Allen  None   None
..         ...   ...    ...
413    Spector  None   None
414      Oliva     y  Ocana
415    Saether  None   None
416       Ware  None   None
417      Peter  None   None

[1309 rows x 3 columns]


In [146]:
temp_surname[temp_surname[2].isna() == False]

,0,1,2
170,Van,der,hoef
307,Penasco,y,Castellana
505,Penasco,y,Castellana
547,Padro,y,Manent
866,Duran,y,More
73,Ovies,y,Rodriguez
220,Duran,y,More
369,Pallas,y,Castello
414,Oliva,y,Ocana


"Vad der hoef" can be loosely translated "from hoef", so it should be treated as surname in our case

The two surnames may be connected by the letter "y" which means "and." For example, Juan Gomez y Conde has Juan as a first name, Gomez as the surname of the father, and Conde as the surname of the mother. Let's check if any of these exist in "Surname" column

In [147]:
y_names = temp_surname[(temp_surname[2].isna() == False) & (temp_surname[1] == 'y')]
df_train[df_train['Surname'] == y_names[2].str]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names


Let's check another column for values not equal None

In [149]:
temp_surname[temp_surname[1].isna() == False]

,0,1,2
18,Vander,Planke,None
38,Vander,Planke,None
153,van,Billiard,None
170,Van,der,hoef
200,Vande,Walle,None
282,de,Pelsmaeker,None
286,de,Mulder,None
307,Penasco,y,Castellana
333,Vander,Planke,None
355,Vanden,Steen,None


Again, let's check to be sure that there isn't person with Surname of a second column

In [150]:
y_names = temp_surname[(temp_surname[1].isna() == False)]
df_train[df_train['Surname'] == y_names[1].str]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names


Let's check non letter values

In [151]:
df_train[~df_train['Surname'].str.isalpha()]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names
18,19,0.0,3,female,31.0,1,0,345763,18.0000,NaN,S,Vander Planke,Mrs,Julius (Emelia Maria Vandemoortele)
28,29,1.0,3,female,NaN,0,0,330959,7.8792,NaN,Q,O'Dwyer,Miss,"Ellen ""Nellie"""
38,39,0.0,3,female,18.0,2,0,345764,18.0000,NaN,S,Vander Planke,Miss,Augusta Maria
39,40,1.0,3,female,14.0,1,0,2651,11.2417,NaN,C,Nicola-Yarred,Miss,Jamila
47,48,1.0,3,female,NaN,0,0,14311,7.7500,NaN,Q,O'Driscoll,Miss,Bridget
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,1272,NaN,3,male,NaN,0,0,366713,7.7500,NaN,Q,O'Connor,Mr,Patrick
394,1286,NaN,3,male,29.0,3,1,315153,22.0250,NaN,S,Kink-Heilmann,Mr,Anton
397,1289,NaN,1,female,48.0,1,1,13567,79.2000,B41,C,Frolicher-Stehli,Mrs,Maxmillian (Margaretha Emerentia Stehli)
398,1290,NaN,3,male,22.0,0,0,347065,7.7750,NaN,S,Larsson-Rondberg,Mr,Edvard A


Let's check dashed surnames which might be surname given at marriage

In [152]:
y_names = df_train[df_train['Surname'].str.contains("-")]['Surname'].str.split('-', expand=True).apply(lambda x: x.str.strip())
y_names

,0,1
39,Nicola,Yarred
49,Arnold,Franchi
125,Nicola,Yarred
184,Kink,Heilmann
192,Andersen,Jensen
351,Williams,Lambert
353,Arnold,Franchi
430,Bjornstrom,Steffansson
444,Johannesen,Bratthammer
587,Frolicher,Stehli


Check if any other person has such surname

In [153]:
df_train[df_train['Surname'] == y_names[0].str]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names


In [154]:
df_train[df_train['Surname'] == y_names[1].str]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names


It seems that nothing else can be done for Surname, let's move to Title column

In [155]:
df_train['Title'].value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Mlle              2
Major             2
Ms                2
Lady              1
Sir               1
Mme               1
Don               1
Capt              1
the Countess      1
Jonkheer          1
Dona              1
Name: Title, dtype: int64

In [156]:
px.bar(x=df_train['Title'].value_counts().index, y=df_train['Title'].value_counts(), template='plotly_dark', title='Title occurrences',
       width=950, color_discrete_sequence=['#03fcb5'])

Ovewhelming part of titles belong to 'Mr', 'Miss', 'Mrs' and 'Master' class. Let's check how big is this part

In [157]:
len(df_train[(df_train['Title'] == 'Mr') | (df_train['Title'] == 'Miss') | (df_train['Title'] == 'Mrs') | (df_train['Title'] == 'Master')]) / len(df_train)

0.974025974025974

Let's check what is percentage of survival based on most common titles

In [174]:
print(f"Mr:   {len(df_train[(df_train['Title'] == 'Mr') & df_train['Survived'] == 1.0]) / len(df_train[(df_train['Title'] == 'Mr') & ~df_train['Survived'].isna()]) }")
print(f"Miss:   {len(df_train[(df_train['Title'] == 'Miss') & df_train['Survived'] == 1.0]) / len(df_train[(df_train['Title'] == 'Miss') & ~df_train['Survived'].isna()]) }")
print(f"Mrs:   {len(df_train[(df_train['Title'] == 'Mrs') & df_train['Survived'] == 1.0]) / len(df_train[(df_train['Title'] == 'Mrs') & ~df_train['Survived'].isna()]) }")
print(f"Master:   {len(df_train[(df_train['Title'] == 'Master') & df_train['Survived'] == 1.0]) / len(df_train[(df_train['Title'] == 'Master') & ~df_train['Survived'].isna()]) }")

Mr:   0.15667311411992263
Miss:   0.6978021978021978
Mrs:   0.792
Master:   0.575


In [158]:
names = df_train[df_train['Names'].str.contains("\\)")]['Names'].str.split(' ', expand=True).apply(lambda x: x.str.strip())
print(names)

     index           0            1           2              3        4     5  \
0        1        John      Bradley   (Florence         Briggs  Thayer)  None   
1        3     Jacques        Heath       (Lily            May    Peel)  None   
2        8       Oscar            W  (Elisabeth     Vilhelmina    Berg)  None   
3        9    Nicholas       (Adele      Achem)           None     None  None   
4       15       (Mary            D   Kingcome)           None     None  None   
..     ...         ...          ...         ...            ...      ...   ...   
216    391      Ernest            H  (Elizabeth        Lindsey   James)  None   
217    395      Lucien       Philip       (Mary         Eloise  Hughes)  None   
218    397  Maxmillian  (Margaretha   Emerentia        Stehli)     None  None   
219    405      Alfred       (Baron         von  Drachstedt")"     None  None   
220    411     William       Edward    (Lillian              E  Thorpe)  None   

        6     7     8     9

In [49]:
names[names[9].isna() == False]

'Vallejo'

In [48]:
df_train[df_train['Surname'] == names.iloc[44, 10].replace(")","")]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names


In [33]:
names[names[8].isna() == False]

,0,1,2,3,4,5,6,7,8,9
307,Victor,de,Satode,(Maria,Josefa,Perez,de,Soto,y,Vallejo)


In [34]:
names[names[7].isna() == False]

,0,1,2,3,4,5,6,7,8,9
307,Victor,de,Satode,(Maria,Josefa,Perez,de,Soto,y,Vallejo)


In [35]:
names[names[6].isna() == False]

,0,1,2,3,4,5,6,7,8,9
307,Victor,de,Satode,(Maria,Josefa,Perez,de,Soto,y,Vallejo)
427,Kate,Florence,"(""Mrs",Kate,Louise,Phillips,"Marshall"")",None,None,None
